# Python Mcity OCTANE Rest API

The Octane API provides REST support for making requests and getting status of various states of the Mcity test facility.

Follow the steps below to test how to utilize the API with Python.
In this example we'll controla a single traffic signal within the facility.

## Connect
First let's load the required libraries and source our connection parameters from the environment. We'll setup a client and then after discussing authentication, connect.

In [2]:
import os
from dotenv import load_dotenv
import requests

#Load environment variables
load_dotenv()
api_key = os.environ.get('MCITY_OCTANE_KEY', None) 
server = os.environ.get('MCITY_OCTANE_SERVER', 'http://localhost:5000')

#If no API Key provided, exit.
if not api_key:
    print ("No API KEY SPECIFIED. EXITING")
    exit()

## Authentication
Calls to the OCTANE REST API are authenticated and authorized by presence of a valid API Key/Token provided in the header of each request. In production this key is provided through email before your reservation. In test environments Mcity staff can provide valid keys for your utilization.

Let's build a header we'll send with each call.

In [3]:
headers = {
    'accept': 'application/json',
    'X-API-KEY': api_key,
    'Content-Type': 'application/json'
}

Next, let's build a request.

For this example we'll set intersection State/Liberty in the North/South direction to be red lights.

First let's query /intersections endpoint to get the ID of this intersection. For the Mvillage test environment this traffic signal/intersection ID is 1.

In [4]:
uri = server + '/api/intersections'
response = requests.get(uri, headers=headers)
print (response.text)

{"intersections":[{"id":1,"instrument":"signal","latitude":42.3003710,"longitude":-83.6986530,"name":"Liberty/State","phases":[{"bound":"south","number":2,"turn":"through","updated":null},{"bound":"north","number":6,"turn":"through","updated":null},{"bound":"west","number":4,"turn":"through","updated":null},{"bound":"east","number":8,"turn":"through","updated":null}],"stage":[{"controlGroup":"00100010","name":"Liberty State South/North"},{"controlGroup":"10001000","name":"Liberty State East/West"}],"state":{"callPedestrian":"00000000","callVehicle":"00000000","forceOff":"00000000","hold":"00000000","omit":"00000000","pedestrianClear":"00000000","phases":[{"callPedestrian":false,"callVehicle":false,"forceOff":false,"hold":false,"omit":false,"omitPedestrian":false,"pedestrianClear":false,"phase":1,"walk":false,"walkDont":true},{"callPedestrian":false,"callVehicle":false,"forceOff":false,"hold":false,"omit":false,"omitPedestrian":false,"pedestrianClear":false,"phase":2,"walk":false,"walkD

## Control an intersection.

Let's set State/Liberty North/South to be a red light.

Now that we know State/Liberty is intersection 1, we next look at the stage data provided for this intersection. Stages are combinations of non-conflicting phases of a light. A phase is best described as one direction of an intersection that is green. If opposing directions are both green, then 2 phases are green. This combination of phases is a stage. The facility API provides lists of valid non-conflicting phases as stages.

The traffic controller at each intersection only lets you set holds on Green. If you wish to hold a light red, you need to set the opposing directions to be green, and hold that. In this intersection West/East is the opposing set of directions and they are represented by phases 4 and 8. To achieve North/South red, we'll hold West/East green.

## Issue commands to change light. 
- reset all settings at this intersection.
- omit the phases we do not want green, stopping them from happening again.
- hold the phases we want to be green (4+8)
- force off the phases we don't want to be green (if they are presently on)

In [5]:
data = '{ "state": { "reset": true, "omit": "01110111", "hold": "10001000", "forceOff": "01110111" }}'

uri = server + '/api/intersection/1'
response = requests.patch(uri, headers=headers, data=data)
print (response.text)

{"requestID":"9d6de4e0-e9c3-4eaf-8ceb-dcb7d007958a"}



The request to change the light was sucessful. The REST API returns a request ID that is trackable using additional API endpoints. To see when your request takes effect you can also query the intersection with a get request. Additionally review the examples using the Socket.IO interface to OCTANE. Socket.IO will push a notice that the request made above was received, and another update when the request has been serviced. The intersection itself will broadcast and update when the light changes to the desired colors.